In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries

In [ ]:
import pandas as pd
import imblearn
import seaborn as sns
sns.set_theme(style="darkgrid")
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict, cross_val_score, KFold, GridSearchCV
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE 

# Loading Data

In [ ]:
df = pd.read_csv('../input/machine-predictive-maintenance-classification/predictive_maintenance.csv')

df.head()

# Exploratory Data Analysis

In [ ]:
df.info()

we can see that there is no missing value in the data. it's a good start !!

In [ ]:
df.nunique()

"UDI" and "ProductID" are unique for each instances, so we can remove them from this data.

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(2,1,1)
pl = sns.countplot(y=df["Type"])
pl.set_title("Type (Frequency)")
pl.set(xlabel=None)
for p in pl.patches:
        percentage = p.get_width()
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        pl.annotate(percentage, (x, y))
plt.subplot(2,1,2)
pl2 = sns.countplot(y=df["Failure Type"])
pl2.set_title("Failure Type (Frequency)")
pl2.set(xlabel=None)
for p in pl2.patches:
        percentage = p.get_width()
        x = p.get_x() + p.get_width() + 0.02
        y = p.get_y() + p.get_height()/2
        pl2.annotate(percentage, (x, y))
plt.show()

### Correlation Matrix

In [ ]:
plt.figure(figsize=(10,10))
pl = sns.heatmap(df.corr(), annot=True)
pl.set_title("Correlation Matrix")
plt.show()

We need to remove:

    1. "Torque [Nm]" or "Rotational speed [rpm]"
    2. "Air temperature [K]" or "Process temperature [K]"

to make this decision, I decided to view compare their data distribution against failure type.

### Data Distribution

In [ ]:
plt.figure(figsize=(10,10))
df_drop = df.drop(columns=['Target','Product ID', 'UDI','Type'])
df_normalize=(df_drop-df_drop.mean())/df_drop.std()
pl=sns.boxplot(data=df_normalize.drop(columns='Failure Type'), orient = 'h')
pl.set_title("Box Plot")
plt.show()